In [1]:
import pandas as pd
import numpy as np
import spacy
nlp = spacy.load("en_core_web_sm")
import json
from spacy.lemmatizer import Lemmatizer, ADJ, NOUN, VERB

lemmatizer = nlp.vocab.morphology.lemmatizer

In [2]:
import operator
fileList=['TextualEntailment']
resultDf=pd.DataFrame()
for num,fileName in enumerate(fileList):
    print(fileName)
    dfs=pd.read_excel("FinalDatasetAuto"+fileName+".xlsx",index_col=0)
    dfs=dfs[427:]
    dfs=dfs.reset_index(drop=True)
    for index,i in enumerate(dfs['adj/noun']):
        if dfs.loc[index,'adj/noun-label']=='N':
            dfs.loc[index,'arg0']=''
            dfs.loc[index,'arg1']=''
            dfs.loc[index,'V']=''
            dfs.loc[index,'baseV']=''
            dfs.loc[index,'PP']=''
    dfs['arg0']=dfs['arg0'].replace(np.NaN,'')
    dfs['arg1']=dfs['arg1'].replace(np.NaN,'')
    dfs['PP']=dfs['PP'].replace(np.NaN,'')
    columnList=['arg0','V','arg1','PP']
    for index,i in enumerate(dfs['adj/noun']):
        result=''
        for column in columnList:
            if dfs.loc[index,column]!='':
                    result=result+dfs.loc[index,column]+" "
        dfs.loc[index,'gold']=result
    for index,i in enumerate(dfs['adj/noun']):
        if index+1<len(dfs['adj/noun']) and dfs.loc[index,'adj/noun']==dfs.loc[index+1,'adj/noun'] and dfs.loc[index,'n_v']==dfs.loc[index+1,'n_v'] and dfs.loc[index,'prep']==dfs.loc[index+1,'prep'] and dfs.loc[index,'pobj']==dfs.loc[index+1,'pobj']:
            dfs.loc[index,'gold2']=dfs.loc[index+1,'gold']
        else:
            dfs.loc[index,'gold2']=dfs.loc[index,'gold']
    for index,i in enumerate(dfs['adj/noun']):
        if dfs.loc[index,'HighestScorePattern']=='M' or dfs.loc[index,'HighestScorePattern']=='OVB' or dfs.loc[index,'HighestScorePattern']=='VOB' or dfs.loc[index,'HighestScorePattern']=='V1' or dfs.loc[index,'HighestScorePattern']=='V2':
            tempDict=dfs.loc[index,'MVOScore':'OVpMPassiveScore'].to_dict()
            tempDict={k: v for k, v in tempDict.items() if str(v) != 'nan' and k!='HighestScorePattern' and k!='MScore' and k!='V1Score' and k!='V2Score'}
            dfs.loc[index,'HighestScorePattern']=max(tempDict.items(), key=operator.itemgetter(1))[0][:-5]
    for index,i in enumerate(dfs['adj/noun']):
        my_dict=dfs.loc[index,'MVOScore':'OVpMPassiveScore'].to_dict()
        my_dict={k: v for k, v in my_dict.items() if str(v) != 'nan' and k!='HighestScorePattern' and k!='MScore' and k!='V1Score' and k!='V2Score'}
        highestScorePatternList=sorted(my_dict, key=my_dict.get, reverse=True)[:3]
        dfs.loc[index,'HighestScorePattern']=",".join(highestScorePatternList)
    dfs.to_excel("FinalDatasetAuto"+fileName+"Top3RemoveArticleMetric.xlsx")

TextualEntailment
